In [1]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Double Peaked SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 13 January 2022
# Last Revision: 11 December 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [4]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2




In [5]:
# Loading in the catalog from Hwang+2020 now...

# Hwang+2020 does not include redshifts in their tables. We will need to query SDSS for those....

hwang2020t2 = ((Table.read('Tables/Hwang2020/table2.dat', readme = 'Tables/Hwang2020/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
# Note here that any cells containing '--' in Xin's table are going to be replaced by 0's

hwang2020t3 = ((Table.read('Tables/Hwang2020/table3.dat', readme = 'Tables/Hwang2020/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

hwang2020 = pd.concat([hwang2020t2,hwang2020t3])

hwang2020['Name1'] = hwang2020['SDSS']
hwang2020['Name2'] = '-99' #hwang2020['SDSS']
hwang2020['z1'] = -99
hwang2020['z2'] = -99
hwang2020['z1_type'] = '-99'
hwang2020['z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(hwang2020,hwang2020['Name1'])

# Converting the coordinates
#coordconvert = SkyCoord(ra = hwang2020['RA'], dec = hwang2020['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=hwang2020['RAdeg']*u.degree, dec=hwang2020['DEdeg']*u.degree, frame='icrs')
hwang2020['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
hwang2020['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)


hwang2020['RA1_deg'] = hwang2020['RAdeg'] #coordconvert.ra.degree
hwang2020['Dec1_deg'] = hwang2020['DEdeg'] #coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
hwang2020['RA2'] = hwang2020['RA1']
hwang2020['Dec2'] = hwang2020['Dec1']

hwang2020['RA2_deg'] = hwang2020['RA1_deg']
hwang2020['Dec2_deg'] = hwang2020['Dec1_deg']

# Adding details about the coordinates
hwang2020['Equinox1'] = "J2000"
hwang2020['Coordinate_waveband1'] = "Optical"
hwang2020['Coordinate_Source1'] = "GAIA DR2 Barycenter Coordinates"

hwang2020['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
hwang2020['Brightness1'] = -100
hwang2020['Brightness_band1'] = -100
hwang2020['Brightness_type1'] = -100

hwang2020['Brightness2'] = -100
hwang2020['Brightness_band2'] = -100
hwang2020['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
hwang2020['Sep'] = -99 # arcseconds
# We do not have angular separations for these objects


#hwang2020['Sep(kpc)'] = hwang2020['Sep']*((cosmo.arcsec_per_kpc_proper(hwang2020['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#hwang2020['delta_z'] = hwang2020['z']-hwang2020['z2']
hwang2020['dV'] = -99 #(2.99e+5)*((1+hwang2020['z'])**2 - (1+hwang2020['z2'])**2)/((1+hwang2020['z'])**2+(1+hwang2020['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
hwang2020['Selection Method'] = "Varstrometry" #DPSELs
hwang2020['Confirmation Method'] = '-99'
hwang2020['Paper(s)'] = "Hwang+2020"
hwang2020['BibCode(s)'] = "2020ApJ...888...73H"
hwang2020['DOI(s)'] = "https://doi.org/10.3847/1538-4357/ab5c1a"

hwang2020['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#hwang2020.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#hwang2020 = hwang2020[['System Type','Name1','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox1','Coordinate_waveband1','Coordinate_Source1','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#hwang2020.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox1','Coordinate_waveband1','Coordinate_Source1','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

hwang2020

,SDSS,RAdeg,DEdeg,plx,Errplx,pm,Errpm,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,010212.54+014032.3,15.552269,1.675662,3.492684,5.955044,NaN,NaN,010212.54+014032.3,-99,-99,-99,-99,-99,1:02:12.54,1:40:32.38,15.552269,1.675662,1:02:12.54,1:40:32.38,15.552269,1.675662,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
1,024634.09-082536.1,41.641862,-8.426549,NaN,NaN,NaN,NaN,024634.09-082536.1,-99,-99,-99,-99,-99,2:46:34.05,-8:25:35.57,41.641862,-8.426549,2:46:34.05,-8:25:35.57,41.641862,-8.426549,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
2,024634.09-082536.1,41.642116,-8.426722,1.630605,6.796264,NaN,NaN,024634.09-082536.1,-99,-99,-99,-99,-99,2:46:34.11,-8:25:36.20,41.642116,-8.426722,2:46:34.11,-8:25:36.20,41.642116,-8.426722,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
3,091402.90+084121.6,138.512081,8.689364,1.607955,5.046658,NaN,NaN,091402.90+084121.6,-99,-99,-99,-99,-99,9:14:02.90,8:41:21.71,138.512081,8.689364,9:14:02.90,8:41:21.71,138.512081,8.689364,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
4,102806.07+502126.1,157.025334,50.357237,2.532618,7.074561,NaN,NaN,102806.07+502126.1,-99,-99,-99,-99,-99,10:28:06.08,50:21:26.05,157.025334,50.357237,10:28:06.08,50:21:26.05,157.025334,50.357237,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
5,114734.05+411928.4,176.891897,41.324538,7.178163,5.585203,NaN,NaN,114734.05+411928.4,-99,-99,-99,-99,-99,11:47:34.06,41:19:28.34,176.891897,41.324538,11:47:34.06,41:19:28.34,176.891897,41.324538,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
6,123204.05+375855.8,188.016914,37.982140,8.151470,5.061317,NaN,NaN,123204.05+375855.8,-99,-99,-99,-99,-99,12:32:04.06,37:58:55.70,188.016914,37.982140,12:32:04.06,37:58:55.70,188.016914,37.982140,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
7,132128.67+541855.5,200.369409,54.315441,4.916714,6.440528,NaN,NaN,132128.67+541855.5,-99,-99,-99,-99,-99,13:21:28.66,54:18:55.59,200.369409,54.315441,13:21:28.66,54:18:55.59,200.369409,54.315441,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
8,132128.67+541855.5,200.369787,54.315397,NaN,NaN,NaN,NaN,132128.67+541855.5,-99,-99,-99,-99,-99,13:21:28.75,54:18:55.43,200.369787,54.315397,13:21:28.75,54:18:55.43,200.369787,54.315397,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,
9,144432.35+602939.4,221.134732,60.494329,7.349597,7.591340,NaN,NaN,144432.35+602939.4,-99,-99,-99,-99,-99,14:44:32.34,60:29:39.58,221.134732,60.494329,14:44:32.34,60:29:39.58,221.134

In [6]:
# Loading in the catalog from Hwang+2020 now...

# Hwang+2020 does not include redshifts in their tables. We will need to query SDSS for those....


orosz2013 = pd.read_csv('Tables/Orosz2013/Orosz2013.csv', sep=',')

orosz2013['Name1'] = orosz2013['Designation']
orosz2013['Name2'] = '-99' #orosz2013['Designation']
orosz2013['z1'] = orosz2013['z']
orosz2013['z2'] = -99
orosz2013['z1_type'] = '-99'
orosz2013['z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(orosz2013,orosz2013['Name1'])

orosz2013['RA1'] = orosz2013['RA']
orosz2013['Dec1'] = orosz2013['Dec']

# Converting the coordinates
coordconvert = SkyCoord(ra = orosz2013['RA'], dec = orosz2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
orosz2013['RA1_deg'] = coordconvert.ra.degree
orosz2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
orosz2013['RA2'] = orosz2013['RA']
orosz2013['Dec2'] = orosz2013['Dec']

orosz2013['RA2_deg'] = orosz2013['RA1_deg']
orosz2013['Dec2_deg'] = orosz2013['Dec1_deg']

# Adding details about the coordinates
orosz2013['Equinox1'] = "J2000"
orosz2013['Coordinate_waveband1'] = "Radio"
orosz2013['Coordinate_Source1'] = "ICRF"

orosz2013['System Type'] = 'Dual AGN Candidate / Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
orosz2013['Brightness1'] = -100
orosz2013['Brightness_band1'] = -100
orosz2013['Brightness_type1'] = -100

orosz2013['Brightness2'] = -100
orosz2013['Brightness_band2'] = -100
orosz2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
orosz2013['Sep'] = (1.e-3)*orosz2013['OpticalOffset_total(mas)'] # arcseconds

#orosz2013['Sep(kpc)'] = orosz2013['Sep']*((cosmo.arcsec_per_kpc_proper(orosz2013['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#orosz2013['delta_z'] = orosz2013['z']-orosz2013['z2']
orosz2013['dV'] = -99 #(2.99e+5)*((1+orosz2013['z'])**2 - (1+orosz2013['z2'])**2)/((1+orosz2013['z'])**2+(1+orosz2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
orosz2013['Selection Method'] = "Radio-Optical Offsets" #DPSELs
orosz2013['Confirmation Method'] = '-99'
orosz2013['Paper(s)'] = "Orosz+2013"
orosz2013['BibCode(s)'] = "2013A&A...553A..13O"
orosz2013['DOI(s)'] = "https://doi.org/10.1051/0004-6361/201321279"

orosz2013['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#orosz2013.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#orosz2013 = orosz2013[['System Type','Name1','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox1','Coordinate_waveband1','Coordinate_Source1','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#orosz2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox1','Coordinate_waveband1','Coordinate_Source1','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

objs = ['J1526-1351']
for index, row in orosz2013.iterrows():
    if row['Name1'] in objs:
        #the_whills.at[index, 'System Type']='Dual AGN'
        orosz2013.at[index, 'Notes']+=' Radio and optical offset could be attributed to the fact that the radio peak is found at the end of the northern jet ; the AGN optical position would then obviously be different.'

objs = ['J1506+3730']
for index, row in orosz2013.iterrows():
    if row['Name1'] in objs:
        orosz2013.at[index, 'Notes']+=' Radio and optical offset could be  attirubuted to dust lane in galaxy.'

# adding notes down here for individual sources:
    #J1526−1351 radio and optical offset could be attributed to the fact that the radio peak is found at the end of the northern jet ; the AGN optical position would then obviously be different
    #J1506+3730 radio and optical offset could be  attirubuted to dust lane in galaxy

    
orosz2013

,Designation,RA,Dec,OpticalOffset_total(mas),OpticalOffset_RA(mas),OpticalOffset_Dec(mas),m_r(mag),z,Type,Ref,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J0038-2120,00:38:29.9547,-21:20:04.023,225,117,192,18.59,0.338,G,NaN,J0038-2120,-99,0.338,-99,-99,-99,00:38:29.9547,-21:20:04.023,9.624811,-21.334451,00:38:29.9547,-21:20:04.023,9.624811,-21.334451,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.225,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
1,J0041+1339,00:41:17.2110,13:39:27.527,348,179,-298,22.53,NaN,G,NaN,J0041+1339,-99,NaN,-99,-99,-99,00:41:17.2110,13:39:27.527,10.321712,13.657646,00:41:17.2110,13:39:27.527,10.321712,13.657646,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.348,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
2,J0106+3402,01:06:00.2934,34:02:02.988,431,428,-52,24.05,0.579,G,NaN,J0106+3402,-99,0.579,-99,-99,-99,01:06:00.2934,34:02:02.988,16.501223,34.034163,01:06:00.2934,34:02:02.988,16.501223,34.034163,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.431,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
3,J0106+2539,01:06:10.9690,25:39:30.496,353,-119,-333,17.41,0.199,G,NaN,J0106+2539,-99,0.199,-99,-99,-99,01:06:10.9690,25:39:30.496,16.545704,25.658471,01:06:10.9690,25:39:30.496,16.545704,25.658471,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.353,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
4,J0106-0315,01:06:43.2287,-03:15:36.296,207,44,-202,17.65,NaN,Q,NaN,J0106-0315,-99,NaN,-99,-99,-99,01:06:43.2287,-03:15:36.296,16.680120,-3.260082,01:06:43.2287,-03:15:36.296,16.680120,-3.260082,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.207,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
5,J0146+2110,01:46:58.7839,21:10:24.381,277,122,-248,23.03,NaN,Q,NaN,J0146+2110,-99,NaN,-99,-99,-99,01:46:58.7839,21:10:24.381,26.744933,21.173439,01:46:58.7839,21:10:24.381,26.744933,21.173439,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.277,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
6,J0216-0118,02:16:05.6638,-01:18:03.397,193,-162,-106,18.73,NaN,Q,NaN,J0216-0118,-99,NaN,-99,-99,-99,02:16:05.6638,-01:18:03.397,34.023599,-1.300944,02:16:05.6638,-01:18:03.397,34.023599,-1.300944,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.193,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
7,J0216-0105,02:16:12.2119,-01:05:18.826,179,-141,-111,17.68,1.492,G,NaN,J0216-0105,-99,1.492,-99,-99,-99,02:16:12.2119,-01:05:18.826,34.050883,-1.088563,02:16:12.2119,-01:05:18.826,34.050883,-1.088563,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.179,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,
8,J0334+0800,03:34:53.3167,08:00:14.419,272,-179,-205,22.49,1.982,G,NaN,J0334+0800,-99,1.982,-99,-99,-99,03:34:53.3167,08:00:14.419,53.722153,8.004005,03:34:53.3167,08:00:14.419,53.722153,8.004005,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.272,-99,Radio-Optical Offsets,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361

In [7]:
#tunique, tmatches, idx1, idx2 = match_tables_fib(hwang2020,orosz2013,5)

# There are no matches between these two tables!


In [8]:
# cooncatenating the two previous frames here...
frodo = pd.concat([hwang2020,orosz2013])

frodo.drop(labels=['SDSS','RAdeg','DEdeg','plx','Errplx','pm','Errpm','Designation','OpticalOffset_total(mas)',\
                   'OpticalOffset_RA(mas)','OpticalOffset_Dec(mas)','m_r(mag)','z','Type','Ref'], axis=1, inplace=True,)


In [9]:
frodo

,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,RA,Dec
0,010212.54+014032.3,-99,-99.000,-99,-99,-99,1:02:12.54,1:40:32.38,15.552269,1.675662,1:02:12.54,1:40:32.38,15.552269,1.675662,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
1,024634.09-082536.1,-99,-99.000,-99,-99,-99,2:46:34.05,-8:25:35.57,41.641862,-8.426549,2:46:34.05,-8:25:35.57,41.641862,-8.426549,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
2,024634.09-082536.1,-99,-99.000,-99,-99,-99,2:46:34.11,-8:25:36.20,41.642116,-8.426722,2:46:34.11,-8:25:36.20,41.642116,-8.426722,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
3,091402.90+084121.6,-99,-99.000,-99,-99,-99,9:14:02.90,8:41:21.71,138.512081,8.689364,9:14:02.90,8:41:21.71,138.512081,8.689364,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
4,102806.07+502126.1,-99,-99.000,-99,-99,-99,10:28:06.08,50:21:26.05,157.025334,50.357237,10:28:06.08,50:21:26.05,157.025334,50.357237,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
5,114734.05+411928.4,-99,-99.000,-99,-99,-99,11:47:34.06,41:19:28.34,176.891897,41.324538,11:47:34.06,41:19:28.34,176.891897,41.324538,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
6,123204.05+375855.8,-99,-99.000,-99,-99,-99,12:32:04.06,37:58:55.70,188.016914,37.982140,12:32:04.06,37:58:55.70,188.016914,37.982140,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
7,132128.67+541855.5,-99,-99.000,-99,-99,-99,13:21:28.66,54:18:55.59,200.369409,54.315441,13:21:28.66,54:18:55.59,200.369409,54.315441,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
8,132128.67+541855.5,-99,-99.000,-99,-99,-99,13:21:28.75,54:18:55.43,200.369787,54.315397,13:21:28.75,54:18:55.43,200.369787,54.315397,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN
9,144432.35+602939.4,-99,-99.000,-99,-99,-99,14:44:32.34,60:29:39.58,221.134732,60.494329,14:44:32.34,60:29:39.58,221.134732,60.494329,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.000,-99,Varstrometry,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,,NaN,NaN


In [10]:
frodo.to_csv('Varstrometry_tables_DR1.csv', sep=',', index=False)
